TODO overall

1. Root directory management in modules
2. Try catch and pytest

In [1]:
import json
import numpy as np
from datetime import datetime
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model

# ML Client
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

In [2]:
# Training utils
from utils.training_utils import (
    fetch_env, 
    fetch_compute_cluster, 
    delete_compute_cluster, 
    upload_file_to_datastore
    )

In [3]:
with open('./config/config.json','r') as f:
    configs=json.loads(f.read())

subscription_id, resource_group, workspace = configs['subscription_id'], configs['resource_group'], configs['workspace_name']

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

ml_client = MLClient(
    credential, subscription_id, resource_group, workspace
)

# Resource management

## Fetch/create environments

In [8]:
training_env = fetch_env(ml_client=ml_client,
env_name="stock-pricing",
version="0.1")
# conda_file_path="./config/training_env.yml")

In [8]:
deployment_env = fetch_env(
    ml_client=ml_client,
    env_name="stock-pricing",
    version="0.2",
    # conda_file_path="./config/deployment_env.yml"
)

In [9]:
f"{training_env.name}:{training_env.version}"

'stock-pricing:0.1'

In [9]:
f"{deployment_env.name}:{deployment_env.version}"

'stock-pricing:0.2'

## Compute cluster management for training

In [10]:
training_compute = fetch_compute_cluster(ml_client, target_name="training-computer2")

Creating a new cpu compute target...


In [ ]:
delete_compute_cluster(ml_client=ml_client, target_name=compute.name)

# Data management - TODO

TODO
1. Scheduled data download using API
2. Scheduled data upload to Azure datastore
3. Tie these two actions together and deleting local files

4. Also think how it can be done basis triggers

## Schedule automatic data download using API

## Uploading from local file

In [6]:
upload_file_to_datastore(
    ml_client=ml_client,
    file_path="./data/ril.csv",
    name="ril",
    version="3",
    tags={"vers":32})

# Triggering training pipeline - TODO

## Trigger training every week - Set up scheduler

1. Scheduled training TBD
2. Also think of how to trigger jobs

In [12]:
# Create a function to get latest data of a ticker

training_data = ml_client.data.get(name="ril", version=3)

In [13]:
from src.training_script import trigger_train

job_name = "test1"

trigger_train(
    ml_client=ml_client,
    job_name=job_name,
    data_asset_uri=training_data.path,
    environment=training_env,
    compute=training_compute
)

Uploading src (0.02 MBs): 100%|██████████| 16604/16604 [00:02<00:00, 8077.99it/s] 




## Register the model

In [17]:
job_name = "test1"
registered_model_name = "mlopsv1"

model = Model(
        path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/outputs/",
        name=registered_model_name,
        description="Model created from test run.",
        type="custom_model",
    )
ml_client.models.create_or_update(model)

Model({'job_name': 'test1', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'mlopsv1', 'description': 'Model created from test run.', 'tags': {}, 'properties': {}, 'id': '/subscriptions/5d2e45e0-cd7b-4338-b279-455fa4a4c42d/resourceGroups/RG/providers/Microsoft.MachineLearningServices/workspaces/AzureMLWorkspace/models/mlopsv1/versions/1', 'Resource__source_path': None, 'base_path': '/Users/anupam/Documents/Codebase/MLOps_stock_prediction', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x1546f5a90>, 'serialize': <msrest.serialization.Serializer object at 0x1546f5d50>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/5d2e45e0-cd7b-4338-b279-455fa4a4c42d/resourceGroups/RG/workspaces/AzureMLWorkspace/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.test1/outputs', 'datastore': None, 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'custom_model'})

In [14]:
ml_client.compute.begin_delete(name=training_compute.name).wait()

NameError: name 'training_compute' is not defined

# Deploy the model - TODO

TODO
1. Find the latest version of model
2. Traffic management

In [10]:
from src.deployment_script import trigger_deployment

In [17]:
endpoint_name="MLOpsv1"

trigger_deployment(
ml_client,
endpoint_name=endpoint_name,
model=ml_client.models.get(name="mlopsv1", version=1),
env = deployment_env,
deployment_details = {"name":"alpha","traffic":100},
)

AttributeError: 'dict' object has no attribute 'name'

In [18]:
deployment_details = {"name":"alpha","traffic":100}
deployment_details["name"]

'alpha'

# Testing the endpoint

In [ ]:
%%writefile inputs.json
{"d1":2663,"d2":2654.4,"d3":2698,"d4":2690,"d5":2698.12}

In [ ]:
endpoint_name="MLOpsv1"

# test the endpoint (the request will route to blue deployment as set above)
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="yellow",
    request_file="inputs.json",
)